In [27]:
def ma(lead, alpha, past_order, inv, on_the_way):
    return int(lead*alpha* np.mean(past_order)) - inv - on_the_way

def maplus(lead, alpha, past_order, inv, on_the_way):
    return int(lead  * (alpha*np.std(past_order) + np.mean(past_order))) - inv - on_the_way

def arima(lead, alpha, past_order, inv, on_the_way):
    p,d,q,t = alpha
    from statsmodels.tsa.arima_model import ARIMA
    model = ARIMA(past_order, order=(p,d,q))
    model_fit = model.fit()
    yhat, _, interval = model_fit.forecast(steps = lead, alpha = t)
    upperbounds = list(map(lambda x: x[1], interval))
    return sum(upperbounds) - inv - on_the_way

def arima_beta(lead, alpha, past_order, inv, on_the_way):
    p,d,q,t = alpha
    from statsmodels.tsa.arima_model import ARIMA
    model = ARIMA(past_order, order=(p,d,q))
    model_fit = model.fit()
    yhat, _, interval = model_fit.forecast(steps = lead, alpha = t)
    upperbounds = list(map(lambda x: x[1], interval))
    return sum(yhat[:-1]) + upperbounds[-1] - inv - on_the_way


def garch(lead, alpha, past_order, inv, on_the_way):
    from arch import arch_model
    am = arch_model(past_order,p=1, o=1, q=1)
    res = am.fit(update_freq=5, disp='off')
    split_date = datetime(2010,1,1)
    forecasts = res.forecast(horizon=1, start=split_date)
    return int(lead*(forecasts.mean.values[0][0] +alpha* forecasts.variance.values[0][0])) - inv- on_the_way

def sarimax(lead, alpha, past_order, inv, on_the_way):
    # alpha = ((p,d,q),(p,d,q,t),alp)
    from pandas import read_csv
    from pandas import datetime
    from matplotlib import pyplot
    from  statsmodels.tsa.statespace.sarimax import SARIMAX
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from datetime import timedelta
    (order,seasonal_order,alp) = alpha
    #print(past_order)
    model = SARIMAX(past_order.values, order =order,seasonal_order=seasonal_order,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
    model_fit = model.fit()
    #pred = model_fit.get_prediction(start=len(past_order),end = len(past_order) + lead-1 )
    pred = model_fit.get_prediction(start=0,end =lead-1 )
    
    pred_ci = pred.conf_int(alpha = alp)
    #print(pred_ci)
    upperbounds = sum(pred_ci["upper y"])
    return int(upperbounds) - inv - on_the_way

def sarimax_buffer(lead, alpha, past_order, inv, on_the_way):
    # alpha = ((p,d,q),(p,d,q,t),alp,beta,gamma)
    from pandas import read_csv
    from pandas import datetime
    from matplotlib import pyplot
    from  statsmodels.tsa.statespace.sarimax import SARIMAX
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from datetime import timedelta
    
    (order,seasonal_order,alp,beta,gamma) = alpha
    model = SARIMAX(past_order.values, order =order,seasonal_order=seasonal_order,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
    model_fit = model.fit()
    pred = model_fit.get_prediction(start=len(past_order),end = len(past_order) + lead-1 )
    pred_mar = pred.conf_int(alpha = beta)
    pred_ci = pred.conf_int(alpha = alp)
    #print(pred_ci)
    upperbounds = pred_ci["upper y"]
    mean = pred_mar["upper y"]
    buffer = sum(upperbounds-mean)/lead
    return int(sum(mean)+gamma*buffer) - inv - on_the_way

In [7]:
def simulator_date(table, alpha, window, func, LEAD = 12, INIT_AMOUNT = 0, pr = True):
    import numpy as np
    import pandas as pd
    from datetime import datetime
    from datetime import timedelta
    
    LEAD_WEEK = timedelta(days = 7*LEAD)
    
    if INIT_AMOUNT == 0:
        INIT_AMOUNT = np.mean(table["OrderQty"]) * LEAD * 1.2  #buffer??
    # Duplicated table
    copy = table.copy()
    copy["Inventory"] = np.zeros(copy.shape[0])
    copy["Inventory"].iloc[window] = INIT_AMOUNT
    copy["ShippedQty"] = np.zeros(copy.shape[0])
    copy["ArrivalQty"] = np.zeros(copy.shape[0])
    copy["RefillQty"] = np.zeros(copy.shape[0])
    
    # Initialization
    refill_schedules = []
    on_the_way = 0
    inv = INIT_AMOUNT
    results = list()
    for day in copy.index[window:]:
        # 1. Arrival
        if refill_schedules != []:
            if refill_schedules[0][0]<= day:
                inv += refill_schedules[0][1]
                copy.loc[day,"ArrivalQty"] = refill_schedules[0][1]
                on_the_way -= refill_schedules[0][1]
                refill_schedules.pop(0)

        # 2. Order
        order= copy.loc[ day, "OrderQty"]

        # 3. Ship
        if inv < order:
            copy.loc[day,"ShippedQty"] = inv
            inv = 0
        else:
            copy.loc[ day, "ShippedQty"] = order
            inv -= order

        # 4. Refill
        past_order = copy["OrderQty"][day-timedelta(days = 7*window):day]
        refill = func(LEAD, alpha, past_order, inv, on_the_way)
        if refill > 0 :
            on_the_way += refill
            copy.loc[day,"RefillQty"] = refill
            refill_schedules.append((day+LEAD_WEEK,refill) )
        
        # 5. Inventory - audit
        copy.loc[ day,"Inventory"] = inv
        
    # Report :
    avg = np.mean(table["OrderQty"])
    std = np.std(table["OrderQty"])
    fulfill = np.mean(copy["ShippedQty"][window:]/copy["OrderQty"][window:])
    rate = sum(copy["ShippedQty"][window:])/sum(copy["OrderQty"][window:])
    maxInv = max(copy["Inventory"][window+LEAD:])
    avgInv = np.mean(copy["Inventory"][window+LEAD:])
    if pr:
        print("Entering simulator with parameter alpha: %s, window: %d, function: %s "%( alpha, window, func.__name__))
        print("Average Order Qty: %d, Standard deviation: %.2f"%(avg, std ))
        print("Fulfillment rate: %.3f" %(fulfill))
        print("Average rate: %.3f" %(rate))
        print("Maximum Inventory occupation: %d" %maxInv)
        print("Average Inventory occupation: %.0f"% avgInv)
    return avg, std, rate, maxInv, avgInv#, copy["Inventory"]

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
def parser(x):
    return datetime.strptime(x,"%Y-%m-%d")
data = pd.read_csv("itemByWeek\\10239ByWeek.csv", parse_dates=[0], index_col=0, date_parser=parser)
data["OrderQty"] = [float(x) for x in data["OrderQty"]]

In [28]:
import warnings
warnings.filterwarnings("ignore")
simulator_date(data, ((2,1,2),(1,1,1,4),0.3),52,sarimax)

Entering simulator with parameter alpha: ((2, 1, 2), (1, 1, 1, 4), 0.3), window: 52, function: sarimax 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 24009
Average Inventory occupation: 15731


(1157.2115384615386, 432.3649041951536, 1.0, 24009.0, 15730.945652173914)

In [13]:
import pandas as pd
import numpy as np
a = pd.Series(data=[1,2,3,4])
b = pd.Series(data=[2,3,4,5])
np.mean(a/b)


0.6791666666666667

In [11]:
sum(a)/sum(b)

0.7142857142857143

In [46]:
simulator_date(data, ((4,1,0),(1,1,1,4),0.3),52,sarimax)

Entering simulator with parameter alpha: ((4, 1, 0), (1, 1, 1, 4), 0.3), window: 52, function: sarimax 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 22856
Average Inventory occupation: 16287


(1157.2115384615386, 432.3649041951536, 1.0, 22856.0, 16287.282608695652)

In [50]:
simulator_date(data, ((4,1,2),(1,1,1,4),0.1,0.5,1.5),52,sarimax_buffer)

Entering simulator with parameter alpha: ((4, 1, 2), (1, 1, 1, 4), 0.1, 0.5, 1.5), window: 52, function: sarimax_buffer 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.963
Maximum Inventory occupation: 20359
Average Inventory occupation: 6373


(1157.2115384615386,
 432.3649041951536,
 0.9633032376384284,
 20359.0,
 6373.280936454849)

In [45]:
simulator_date(data, ((4,1,0),(1,1,1,4),0.05,0.5,1.5),52,sarimax_buffer)

Entering simulator with parameter alpha: ((4, 1, 0), (1, 1, 1, 4), 0.05, 0.5, 1.5), window: 52, function: sarimax_buffer 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.985
Maximum Inventory occupation: 16741
Average Inventory occupation: 6419


(1157.2115384615386,
 432.3649041951536,
 0.984949213514848,
 16741.0,
 6419.413043478261)

In [31]:
simulator_date(data, (4,1,0,0.30), 50, arima)

Entering simulator with parameter alpha: (4, 1, 0, 0.3), window: 50, function: arima 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.992
Maximum Inventory occupation: 15221
Average Inventory occupation: 6944


(1157.2115384615386,
 432.3649041951536,
 0.991941209807737,
 15221.751709283968,
 6944.3288956167735)

In [40]:
simulator_date(data, 1.3, 26, ma)

Entering simulator with parameter alpha: 1.3, window: 26, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.993
Maximum Inventory occupation: 8808
Average Inventory occupation: 5125


(1157.2115384615386,
 432.3649041951536,
 0.9933265170774842,
 8808.0,
 5125.169491525424)

In [36]:
simulator_date(data, 0.8, 50, maplus)

Entering simulator with parameter alpha: 0.8, window: 50, function: maplus 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.997
Maximum Inventory occupation: 8661
Average Inventory occupation: 4709


(1157.2115384615386,
 432.3649041951536,
 0.9974915013796742,
 8661.0,
 4708.712765957447)